In [1]:
# Import the libraries and modules
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [2]:
# Base URL and headers
BASE_URL = "https://www.cityu.edu.hk/hktech300/start-ups/all-start-ups?page="

# Chrome options
OPTIONS = Options()
OPTIONS.add_argument("--start-maximized")

# Path to ChromeDriver
SERVICE = Service("/opt/homebrew/bin/chromedriver")

In [3]:
# Storage for company data
startup_data = []

# Total pages to scrape
total_pages = 42

In [ ]:
# Initialize WebDriver
driver = webdriver.Chrome(service=SERVICE, options=OPTIONS)

try:
    for page in range(total_pages + 1):
        driver.get(f"{BASE_URL}{page}")

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".card.fund.team"))
            )
        except:
            print("Timeout: Cards not loaded for Page {page}. Skipping...")
            continue

        cards = driver.find_elements(By.CSS_SELECTOR, ".card.fund.team")

        for card in cards:
            try:
                links = card.find_elements(By.TAG_NAME, "a")
                if len(links) >= 2:
                    company_name = links[1].text.strip()
                    company_url = links[1].get_attribute("href").strip()
                    startup_data.append({
                        "Company Name": company_name,
                        "CityU URL": company_url
                    })
                else:
                    print("Incomplete card: <a> tags missing")
            except Exception as e:
                print(f"Error parsing card: {e}")

        time.sleep(1)

finally:
    driver.quit()

Timeout: Cards not loaded. Skipping...


In [5]:
# Create a DataFrame from the collected data
df = pd.DataFrame(startup_data)

In [ ]:
# Save to CSV
csv_file = "hktech300_startups.csv"
df.to_csv(csv_file, index=False, encoding="utf-8")

print(f"{len(df)} startups saved to {csv_file}")


📄✅ Finished! 756 startups saved to hktech300_startups.csv
